## Autoregressive Models

In [ ]:
import warnings

import matplotlib.pyplot as plt 
import pandas as pd
import plotly.express as px 
from pymongo import MongoClient
from sklearn.metrics import  median_absolute_error
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.ar_model import AutoReg

